拼多多2019笔试题。给若干任务，每个任务有完成时间与依赖。现限制单线程工作，求一个使得平均响应时间最小的完成方案。

思路：拓扑排序的变种。

In [ ]:
import sys

n, m = map(int, sys.stdin.readline().strip().split())
d_arr = [0] * n  # 入度数组
adj_table = {job_idx: list() for job_idx in range(n)}
ts = list(map(int, sys.stdin.readline().strip().split()))
for _ in range(m):
    # from, to
    f, t = map(int, sys.stdin.readline().strip().split())
    if f - 1 not in adj_table:
        adj_table[f - 1] = list()
    adj_table[f - 1].append(t - 1)
    d_arr[t - 1] += 1


def func(n, d_arr, ts, adj_table):
    q = list()
    t = 0
    res = list()

    # 找到第一个入度为0且所需时间最少的任务
    cur_min = 0x7FFFFFFF
    cur_job = -1
    for job_idx, d_in in enumerate(d_arr):
        if d_in == 0 and ts[job_idx] < cur_min:
            cur_job, cur_min = job_idx, ts[job_idx]
    q.append(cur_job)

    # 完成n个任务
    for epoch in range(n):
        cur_job = q.pop(0)
        d_arr[cur_job] -= 1
        res.append(cur_job+1)

        if epoch < n:
            for post_job in adj_table[cur_job]:
                d_arr[post_job] -= 1

            bak_job = -1
            cur_min = 0x7FFFFFFF
            for job_idx, d_in in enumerate(d_arr):
                if d_in == 0 and ts[job_idx] < cur_min:
                    bak_job, cur_min = job_idx, ts[job_idx]
            q.append(bak_job)

    return res


print(' '.join(map(str, func(n, d_arr, ts, adj_table))))